<a href="https://colab.research.google.com/github/Katinder/Deep-Learning-Mini-Projects/blob/main/LSTM_SA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Sentiment analysis using Keras imdb dataset

In [1]:
from keras.datasets import imdb

In [2]:
(x_train,y_train),(x_test,y_test)=imdb.load_data()

17473536/17464789 [==============================] - 0s 0us/step


The reviews in this dataset are already pre-processed and encoded into work indexes (default indexing is done on the basis of word-frequency, more the frequency smaller the index)

#### preprocess

In [30]:
import keras
from keras import backend as k
from keras.layers import LSTM, Embedding, Activation, Dense
from keras.models import Sequential

In [31]:
top_words=500
(x_train,y_train),(x_test,y_test)=imdb.load_data(num_words=top_words)

In [32]:
print(x_train[0])
print(y_train[0])

[1, 14, 22, 16, 43, 2, 2, 2, 2, 65, 458, 2, 66, 2, 4, 173, 36, 256, 5, 25, 100, 43, 2, 112, 50, 2, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 2, 2, 17, 2, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2, 19, 14, 22, 4, 2, 2, 469, 4, 22, 71, 87, 12, 16, 43, 2, 38, 76, 15, 13, 2, 4, 22, 17, 2, 17, 12, 16, 2, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2, 2, 16, 480, 66, 2, 33, 4, 130, 12, 16, 38, 2, 5, 25, 124, 51, 36, 135, 48, 25, 2, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 2, 15, 256, 4, 2, 7, 2, 5, 2, 36, 71, 43, 2, 476, 26, 400, 317, 46, 7, 4, 2, 2, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 2, 88, 12, 16, 283, 5, 16, 2, 113, 103, 32, 15, 16, 2, 19, 178, 32]
1


In [33]:
len(x_train)

25000

In [34]:
print(len(x_train[1]))
print(len(x_train[20]))
print(len(set(y_train)))

189
129
2


In [35]:
#pad to make every review the same lenght
from keras.preprocessing import sequence
max_len=500
x_train=sequence.pad_sequences(x_train,maxlen=max_len)
x_test=sequence.pad_sequences(x_test,maxlen=max_len)

In [36]:
print(len(x_train[1]),len(x_train[20]))

500 500


In [74]:
#dictionary to convert back
word_to_id=keras.datasets.imdb.get_word_index()
for x in list(word_to_id)[0:5]:
  print(x, word_to_id[x])

#shift by one
word_to_id={k:(v+1) for k,v in word_to_id.items()}
for x in list(word_to_id)[0:5]:
  print(x, word_to_id[x])

# add 0,1,2
word_to_id["<PAD>"]=0
word_to_id["<START>"]=1

#reverse dictionary
id_to_word={val:key for key,val in word_to_id.items()}
for x in list(id_to_word)[0:5]:
  print(x, id_to_word[x])

fawn 34701
tsukino 52006
nunnery 52007
sonja 16816
vani 63951
fawn 34702
tsukino 52007
nunnery 52008
sonja 16817
vani 63952
34702 fawn
52007 tsukino
52008 nunnery
16817 sonja
63952 vani


In [71]:
print(x_train[0])

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   

In [72]:
id_to_word[14]

'was'

In [75]:
print(' '.join(id_to_word[id] for id in x_train[0] ))

<PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD

#### Model

In [77]:
embedding_vector_length=32
model=Sequential()
model.add(Embedding(top_words,embedding_vector_length, input_length=max_len))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid')) #binary classification
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 32)           16000     
                                                                 
 lstm (LSTM)                 (None, 100)               53200     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 69,301
Trainable params: 69,301
Non-trainable params: 0
_________________________________________________________________


#### training

In [80]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=2,batch_size=128)

Epoch 1/2
196/196 [==============================] - 301s 2s/step - loss: 0.5832 - accuracy: 0.6830 - val_loss: 0.4794 - val_accuracy: 0.7743
Epoch 2/2
196/196 [==============================] - 260s 1s/step - loss: 0.5050 - accuracy: 0.7664 - val_loss: 0.5451 - val_accuracy: 0.7297


In [81]:
model.save('imdb.h5')

#### evaluate

In [82]:
scores=model.evaluate(x_test,y_test,verbose=0)
print("Accuracy: ",scores[1]*100)
print("Loss: ",scores[0])

Accuracy:  72.97199964523315
Loss:  0.5451398491859436


#### Check for input

In [88]:
import numpy as np

r1="it was really fun"
r2="not fun"

for review in [r1,r2]:
  tmp=[]
  for w in review.split(" "):
    tmp.append(word_to_id[w])
  tmp_padded=sequence.pad_sequences([tmp],maxlen=max_len)
  print(f"{review}: {model.predict(np.array([tmp_padded][0]))}")


it was really fun: [[0.19110912]]
not fun: [[0.39025202]]
